In [27]:
import typing

import pandas as pd

test_set = pd.read_json('../vul4c_dataset/test.json')
train_set = pd.read_json('../vul4c_dataset/train.json')
train_set = train_set[
    (train_set['func'].str.split('\n').str.len() < 30) & (train_set['func'].str.split('\n').str.len() > 10)]

In [28]:
vul_train_set = train_set[train_set['vul'] == 1]
non_vul_train_set = train_set[train_set['vul'] == 0]

vul_test_set = test_set[test_set['vul'] == 1]
non_vul_test_set = test_set[test_set['vul'] == 0]

# test_set.sample(n=6 , random_state=2023)
small_vul_test_set = vul_test_set.sample(n= 10, random_state=2023).to_dict('records')
small_non_vul_test_set = non_vul_test_set.sample(n= 20, random_state=2023).to_dict('records')

In [81]:

# prompt_prefix = """I want you to act as a vulnerability detector, your objective is to detect if a function is vulnerable,uch as input validation issues, buffer overflow vulnerabilities, SQL injection vulnerabilities, and other security risks. I will give you a function, answer "yes" if the function is vulnerable, answer "no" if there is no vulnerability, no other information needs to be output.
# """

# icl_random_1
prompt_prefix = """I want you to act as a vulnerability detector, your objective is to detect if a function is vulnerable. I will give you several examples, each containing a function and an answer corresponding to whether there is a vulnerability. At the end I will give you a function that answers "yes" if the function has a vulnerability and "no" if there is no vulnerability, without outputting any other information."""

# icl_random_2
# prompt_prefix = """I want you to act as a vulnerability detector, your objective is to detect if a function is vulnerable. I will give you several examples, each containing a function and an answer corresponding to whether there is a vulnerability. At the end I will give you a function,you should analyze its code for potential security vulnerabilities, such as input validation issues, buffer overflow vulnerabilities, SQL injection vulnerabilities, and other security risks. Answers "yes" if the last function has a vulnerability and "no" if there is no vulnerability, without outputting any other information.
# """


icl_random_vul_train_set = vul_train_set.sample(n=3, random_state=7077)
icl_random_non_vul_train_set = non_vul_train_set.sample(n=3, random_state=7077)
icl_random_set = pd.concat([icl_random_vul_train_set, icl_random_non_vul_train_set]).sample(frac=1,random_state=8080).to_dict('records')
icl_random_vul_info = [
    "",
    "Use After Free",
    "",
    "Use After Free",
    "",
    "NULL Pointer Dereference",
]

icl_random_prompt = ""
for idx,item in enumerate(icl_random_set):
    print(item['cwe_list'])
    print(item['cve'])
    print(item['commit_hash'])
    vulnerable_info = ""
    # if len(icl_random_vul_info[idx]) != 0:
    #     vulnerable_info = f"# vulnerability:{icl_random_vul_info[idx]}\n"
    icl_random_prompt += f"# function\n{item['func']}\n{vulnerable_info}# vulnerable:{'yes' if item['vul'] == 1 else 'no'}\n\n"

print(icl_random_prompt)

def icl_find_same_repo_examples(e:dict):
    repo_name = e['repo_name']
    e_id = e['id']
    vul = vul_train_set[(vul_train_set['repo_name'] == repo_name) & (vul_train_set['id'] != e_id) ]
    non_vul = non_vul_train_set[(non_vul_train_set['repo_name'] == repo_name) & (non_vul_train_set['id'] != e_id)]
    vul = vul.sample(n=3 if vul.shape[0] > 3 else vul.shape[0],random_state=2023)
    non_vul = non_vul.sample(n=3 if non_vul.shape[0] > 3 else non_vul.shape[0],random_state=2023)
    return pd.concat([vul , non_vul]).sample(frac=1,random_state=2023).to_dict('records')


cot_few_shot_info = [
    ( 11299 , """Let's think step by step
1. The function decode_data takes two parameters: a pointer to a structure called struct sixpack named sp, and an unsigned char named inbyte.
2. then it declares a pointer variable buf of type unsigned char.
3. It checks if the rx_count member variable of the sp structure is not equal to 3. If it is not equal to 3, it means there is still space in the raw_buf array to store data.
4. Then it decoded 3 bytes and store the decode result in cooked_buf.
5. Here's a vulnerability that causes out-of-bounds memory writes, the function should check cooked_buf array size""" ),
    (88703 , """Let's think step by step
1. The function addPeer is a member function of the Logger class.
2. It takes three parameters: ip (of type QString) representing the IP address of the peer, blocked (of type bool) indicating if the peer is blocked, and reason (of type QString) providing the reason for blocking.
3. The ip parameter is passed in by the webUI without escaping many values, which could potentially lead to XSS."""),
    (17002 , """Let's think step by step
1. The function dissector_get_string_handle takes two parameters: sub_dissectors: A dissector_table_t variable representing a table of sub-dissectors. string: A const gchar* representing a string.
2. Inside the function callsfind_string_dtbl_entry with the parameters sub_dissectors and string to search for a matching entry in the sub_dissectors table based on the provided string.
3. The result of the find_string_dtbl_entry function is stored in the dtbl_entry variable.
4. if dtbl_entry return the value of dtbl_entry->current else return null
5. There is a vulnerability where table searches find_string_dtbl_entry for empty strings are not handled properly, which allows remote attackers to cause a denial of service."""),
        ( 228194 , """Let's think step by step
1. The function free_wininfo is defined, which takes a pointer to a structure wininfo_T called wip as its parameter.
2. The function first checks if the wi_optset member of the wininfo_T structure pointed to by wip is true using the if statement.
3. If wi_optset is true, the function calls the clear_winopt function, passing the wi_opt member of the wininfo_T structure pointed to by wip as an argument. This function is responsible for clearing the window options.
4. Next, there is an #ifdef directive that checks if the FEAT_FOLDING feature is defined. If it is defined, the code inside the #ifdef block is compiled.
5. Inside the #ifdef FEAT_FOLDING block, the function calls the deleteFoldRecurse function, passing the wi_folds member of the wininfo_T structure pointed to by wip as an argument. This function is responsible for deleting any existing folding information associated with the window.
6. After the #ifdef block, the function calls vim_free to deallocate the memory pointed to by wip.
7. This function is clearly not vulnerable""") ,
    ( 92053, """Let's think step by step
1. The function napi_watchdog is defined, which takes a pointer to a struct hrtimer as an argument and returns an enum value of type hrtimer_restart.
2. The line napi = container_of(timer, struct napi_struct, timer); uses the container_of macro to obtain a pointer to the parent structure struct napi_struct using the timer pointer. It assumes that the timer is embedded within a larger structure struct napi_struct.
3. The next block of code checks certain conditions before scheduling the NAPI (New API) processing.
4. Finally, the function returns HRTIMER_NORESTART, indicating that the high-resolution timer should not be restarted.
5. This function is clearly not vulnerable""" ),
    ( 123609, """Let's think step by step
1. The function atl2_watchdog takes an unsigned long data as a parameter.
2. It checks if the __ATL2_DOWN bit is not set in the adapter flags using the test_bit function.
3. If the bit is not set, it proceeds with the following steps:
4. It acquires a spin lock using spin_lock_irqsave to protect the adapter stats.
5. It reads the value from the register REG_STS_RXD_OV using ATL2_READ_REG macro and assigns it to drop_rxd.
6. It releases the spin lock using spin_unlock_irqrestore.
7. It increments the rx_over_errors counter in the statistics of the adapter's associated network device (netdev) by adding the values of drop_rxd and drop_rxs.
8. Resets the watchdog timer for the adapter by modifying the expiration time using mod_timer
9. This function is clearly not vulnerable""" ),
]



cot_few_shot_str = ""
def add_cot_few_shot_str(item):
    global cot_few_shot_str
    idx , info = item
    example = train_set[train_set['id'] == idx].iloc[0]
    vul = example['vul']
    func = example['func']
    cot_few_shot_str += f'# function\n{func}\n{info}\n# vulnerable:{"yes" if vul == 1 else "no"}\n\n\n'

for i in cot_few_shot_info:
    add_cot_few_shot_str(i)


def write_example(f:typing.IO,e:dict):
    # CoT
    # prompt =  f"{prompt_prefix}\n{e['func']}\n\nlet's think step by step first.\n\n"
    # f.write(prompt)
    # f.write("""answer "yes" if the function is vulnerable, answer "no" if there is no vulnerability, no other information needs to be output.""")
    # f.write('\n\n')

    # CoT few-shot
    prompt =  f"{prompt_prefix}\n{cot_few_shot_str}\n# function\n{e['func']}\nLet's think step by step.\n\n"
    f.write(prompt)
    f.write("""answer "yes" if the function is vulnerable, answer "no" if there is no vulnerability, no other information needs to be output.""")
    f.write('\n\n')

    # ICL random
    # prompt = f"{prompt_prefix}\n{icl_random_prompt}\n{e['func']}\n# vulnerable:"
    #
    # f.write(prompt)
    # f.write('\n\n')

    # ICL select from same repo
    # examples = icl_find_same_repo_examples(e)
    # example_prompt = ""
    # for item in examples:
    #     example_prompt += f"# function\n{item['func']}\n{vulnerable_info}# vulnerable:{'yes' if item['vul'] == 1 else 'no'}\n\n"
    # prompt = f"{prompt_prefix}\n{example_prompt}\n{e['func']}\n# vulnerable:"
    #
    # f.write(prompt)
    # f.write('\n\n')

with open('vul_test.txt',mode='w') as f:
    for e in small_vul_test_set:
        write_example(f,e)



with open('non_vul_test.txt',mode='w') as f:
    for e in small_non_vul_test_set:
        write_example(f,e)



['CWE-416']
CVE-2022-3591
8f3c3c6cd044e3b5bf08dbfa3b3f04bb3f711bad
CWE-Other
CVE-2015-6780
f2cba0d13b3a6d76dedede66731e5ca253d3b2af
['CWE-200']
CVE-2016-2117
f43bfaeddc79effbf3d0fcb53ca477cca66f3db8
CWE-416
CVE-2022-0156
9f1a39a5d1cd7989ada2d1cb32f97d84360e050f
['CWE-476']
CVE-2018-7191
0ad646c81b2182f7fa67ec0c8c825e0ee165696d
CWE-476
CVE-2022-1671
ff8376ade4f668130385839cef586a0990f8ef87
# function
void
free_wininfo(wininfo_T *wip)
{
    if (wip->wi_optset)
    {
	clear_winopt(&wip->wi_opt);
#ifdef FEAT_FOLDING
	deleteFoldRecurse(&wip->wi_folds);
#endif
    }
    vim_free(wip);
}
# vulnerable:no

# function
ConnectionInfoPopupAndroid::ConnectionInfoPopupAndroid(
    JNIEnv* env,
    jobject java_website_settings_pop,
    WebContents* web_contents) {
  // Important to use GetVisibleEntry to match what's showing in the omnibox.
  content::NavigationEntry* nav_entry =
      web_contents->GetController().GetVisibleEntry();
  if (nav_entry == NULL)
    return;

  popup_jobject_.Reset(env, 

In [50]:
# vul_train_set[]
new_vul_train_set = vul_train_set
new_vul_train_set['cwe_list'] = vul_train_set.cwe_list.apply(lambda x: x[0])
new_vul_train_set

/tmp/ipykernel_22150/1942265450.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_vul_train_set['cwe_list'] = vul_train_set.cwe_list.apply(lambda x: x[0])


,cve,cwe_list,repo_name,commit_hash,git_url,func,graph_idx,vul,id
24,CVE-2012-0036,CWE-89,bagder/curl,75ca568fa1c19de4c5358fed246686de8467c238,https://github.com/bagder/curl/commit/75ca568f...,static CURLcode pop3_parse_url_path(struct con...,24450,1,24
42,CVE-2013-6401,CWE-310,akheron/jansson,8f80c2d83808150724d31793e6ade92749b1faa4,https://github.com/akheron/jansson/commit/8f80...,"void *hashtable_get(hashtable_t *hashtable, co...",63402,1,42
59,CVE-2012-0056,CWE-264,torvalds/linux,e268337dfe26dfc7efd422a804dbb27977a3cccc,http://git.kernel.org/?p=linux/kernel/git/torv...,struct mm_struct *mm_for_maps(struct task_stru...,24103,1,59
98,CVE-2023-0216,CWE-476,openssl,934a04f0e775309cadbef0aa6b9692e1b12a76c6,https://git.openssl.org/gitweb/?p=openssl.git;...,void ossl_pkcs7_resolve_libctx(PKCS7 *p7)\n{\n...,423122,1,98
130,CVE-2023-28101,CWE-116,flatpak,6cac99dafe6003c8a4bd5666341c217876536869,https://github.com/flatpak/flatpak/commit/6cac...,"static void\nprint_perm_line (int idx,\...",427327,1,130
...,...,...,...,...,...,...,...,...,...
309353,CVE-2021-3623,CWE-787,stefanberger/libtpms,2e6173c,https://github.com/stefanberger/libtpms/commit...,"TPM_RC\nTPMA_CC_Unmarshal(TPMA_CC *target, BYT...",362924,1,309353
309546,CVE-2020-11049,CWE-125,FreeRDP,c367f65d42e0d2e1ca248998175180aa9c2eacd0,https://github.com/FreeRDP/FreeRDP/commit/c367...,"int rdp_recv_out_of_sequence_pdu(rdpRdp* rdp, ...",289409,1,309546
309566,CVE-2016-6871,CWE-190,facebook/hhvm,c00fc9d3003eb06226b58b6a48555f1456ee2475,https://github.com/facebook/hhvm/commit/c00fc9...,"static String HHVM_FUNCTION(bcmul, const Strin...",161279,1,309566
309587,CVE-2018-7323,CWE-834,wireshark,5d45b69b590cabc5127282d1ade3bca1598e5f5c,https://github.com/wireshark/wireshark/commit/...,static gint\ndissect_wccp2_alternate_mask_assi...,210621,1,309587


In [62]:
new_vul_train_set[new_vul_train_set['cwe_list'] == 'CWE-20']

,cve,cwe_list,repo_name,commit_hash,git_url,func,graph_idx,vul,id
1228,CVE-2010-4818,CWE-20,xorg/xserver,6c69235a9dfc52e4b4e47630ff4bab1a820eb543,http://cgit.freedesktop.org/xorg/xserver/commi...,int __glXDispSwap_QueryServerString(__GLXclien...,34464,1,1228
1675,CVE-2015-3150,CWE-20,abrt/libreport,1951e7282043dfe1268d492aea056b554baedb75,https://github.com/abrt/libreport/commit/1951e...,static void dd_unlock(struct dump_dir *dd)\n{\...,281082,1,1675
3679,CVE-2016-9395,CWE-20,mdadams/jasper,d42b2388f7f8e0332c846675133acea151fc557a,https://github.com/mdadams/jasper/commit/d42b2...,"void jas_matrix_divpow2(jas_matrix_t *matrix, ...",169595,1,3679
5069,CVE-2010-4818,CWE-20,xorg/xserver,ec9c97c6bf70b523bc500bd3adf62176f1bb33a4,http://cgit.freedesktop.org/xorg/xserver/commi...,int __glXDisp_CreateNewContext(__GLXclientStat...,34549,1,5069
7541,CVE-2016-1661,CWE-20,chromium,f23b8e77a83a5aafabf64acf723cf2ac02c5cf0e,https://chromium.googlesource.com/chromium/src...,static bool canAccessFrame(v8::Isolate* isolat...,118234,1,7541
...,...,...,...,...,...,...,...,...,...
298735,CVE-2020-26270,CWE-20,tensorflow,14755416e364f17fb1870882fa778c7fec7f16e3,https://github.com/tensorflow/tensorflow/commi...,static port::StatusOr<CudnnRnnSequenceTensorDe...,308612,1,298735
298867,CVE-2015-3150,CWE-20,abrt,6e811d78e2719988ae291181f5b133af32ce62d8,https://github.com/abrt/abrt/commit/6e811d78e2...,static bool allowed_problem_dir(const char *di...,281041,1,298867
300227,CVE-2010-4575,CWE-20,chromium,eb68c596f0883a539386eed7a11c8a3da0aa529a,https://github.com/chromium/chromium/commit/eb...,void ThemeInstalledInfoBarDelegate::Observe(\n...,17853,1,300227
302145,CVE-2010-5110,CWE-20,poppler,fc071d800cb4329a3ccf898d7bf16b4db7323ad8,http://cgit.freedesktop.org/poppler/poppler/co...,static boolean str_fill_input_buffer(j_decompr...,73208,1,302145


In [64]:
print(new_vul_train_set[new_vul_train_set['id'] == 17002]['func'].iloc[0])

dissector_handle_t
dissector_get_string_handle(dissector_table_t sub_dissectors,
			    const gchar *string)
{
	dtbl_entry_t *dtbl_entry;

	dtbl_entry = find_string_dtbl_entry(sub_dissectors, string);
	if (dtbl_entry != NULL)
		return dtbl_entry->current;
	else
		return NULL;
}


In [73]:
print(icl_random_non_vul_train_set.iloc[2]['func'])

static void atl2_watchdog(unsigned long data)
{
	struct atl2_adapter *adapter = (struct atl2_adapter *) data;

	if (!test_bit(__ATL2_DOWN, &adapter->flags)) {
		u32 drop_rxd, drop_rxs;
		unsigned long flags;

		spin_lock_irqsave(&adapter->stats_lock, flags);
		drop_rxd = ATL2_READ_REG(&adapter->hw, REG_STS_RXD_OV);
		drop_rxs = ATL2_READ_REG(&adapter->hw, REG_STS_RXS_OV);
		spin_unlock_irqrestore(&adapter->stats_lock, flags);

		adapter->netdev->stats.rx_over_errors += drop_rxd + drop_rxs;

		/* Reset the timer */
		mod_timer(&adapter->watchdog_timer,
			  round_jiffies(jiffies + 4 * HZ));
	}
}
